# 模糊综合评价法 (Fuzzy Comprehensive Evaluation, FCE)

模糊综合评价法是一种基于**模糊数学**的综合评价方法。它将“非黑即白”的硬性指标转化为“隶属度”的柔性描述，能够很好地解决现实世界中大量存在的**模糊性**和**难以量化**的问题（如：“好”与“坏”的界限，“高”与“矮”的过渡）。

## 方法核心
传统的精确评价方法要求指标有明确的边界。但在实际场景中，比如评价一个人的“身材”，身高 179cm 和 180cm 真的有本质区别吗？

FCE 通过引入**隶属度（Membership Degree）**，承认事物属性的中间过渡状态。它不是直接给出一个分数，而是给出一个**评价向量**（例如：30% 属于优，50% 属于良，20% 属于差），最后再通过运算得出一个综合结论。

## 数学图景与算例设定

为了演示 FCE 的全过程，我们设定如下的具体评价场景：

### 1. 评价对象与数据集
我们需要评价 3 位候选人（**George, Marx, Anna**）的综合身体素质。
原始观测数据矩阵 $X$ 如下（行代表样本，列代表指标）：

$$
X = 
\begin{bmatrix}
9 & 3 & 4 & 7 \\
6 & 3 & 4 & 6 \\
4 & 9 & 7 & 3 
\end{bmatrix}
\begin{matrix}
\text{(George)} \\ \text{(Marx)} \\ \text{(Anna)}
\end{matrix}
$$

### 2. 因素集 (Criteria) $U$
我们选取 4 个评价指标：
- $u_1$: 体重 (Weight)
- $u_2$: 臂展 (Length)
- $u_3$: 身高 (Height)
- $u_4$: 年龄 (Age)

**假设：** 本例中所有指标均为 **效益型 (Type='max')**，即数值越大，我们认为表现越“突出”或符合某种特质（为了简化计算演示）。

### 3. 评语集 (Grades) $V$
我们设定 4 个等级来描述状态：
$$V = \{g_1, g_2, g_3, g_4\}$$
(例如：极强、强、中、弱)

### 4. 权重向量 (Weights) $A$
不同指标的重要性不同。此处我们直接输入**已通过科学方法（如熵权法或专家打分）确定的权重**：
$$A = [0.10, 0.20, 0.30, 0.40]$$

## Step 1: 构建模糊隶属度矩阵 $R$

这是 FCE 中最关键的一步。我们需要建立从“原始数据”到“评语等级”的映射关系。
对于第 $i$ 个方案的第 $j$ 个指标 $x_{ij}$，它属于第 $k$ 个等级的隶属度记为 $r_{ijk}$。

### 隶属度的来源
在实际应用中，确定隶属度主要有以下两种方法：

1.  **统计试验法（主观/问卷法）**：
    * **适用场景**：定性指标（如“服务态度”、“口味”）。
    * **操作**：组织 10 位专家进行投票。如果对于 George 的“体重”指标，有 8 位专家认为属于 $g_1$（极强），2 位认为属于 $g_2$，则隶属度向量为 $[0.8, 0.2, 0, 0]$。
    
2.  **模糊分布函数法（客观/数学法）**：
    * **适用场景**：定量指标（如本例中的身高、长度数值）。
    * **操作**：利用数学函数（如**梯形分布**、**三角形分布**  或 **高斯分布**）计算隶属度。
    * *注：本算法包在代码中采用了这种方法，根据数据的分布自动生成参数，计算每个具体数值落在不同等级区间的“概率”。*

最终，对于每一个样本，我们都会得到一个模糊关系矩阵 $R_i$（维度：指标数 $m \times$ 等级数 $k$）。

## Step 2: 模糊合成 (Fuzzy Synthesis)

得到权重向量 $A$ 和隶属度矩阵 $R$ 后，我们需要将它们“合成”，得到最终的模糊评价向量 $B$。
公式为：
$$B = A \circ R$$

### 常见的合成算子
“$\circ$” 代表模糊合成运算。根据业务逻辑的严苛程度，有多种算子可供选择：

1.  **加权平均型 $M(\cdot, +)$ —— 这里的首选**
    * **逻辑**：普通的矩阵乘法。$b_k = \sum_{j=1}^{n} a_j \cdot r_{jk}$。
    * **特点**：综合考虑所有因素的影响，保留信息最完整。
    * *注：本算法包的主函数**默认采用**此算子，利用 `np.dot` 或 `np.tensordot` 实现。*

2.  **主因素决定型 $M(\land, \lor)$ (扎德算子 / Zadeh Operator)**
    * **逻辑**：先取小，后取大。$b_k = \max_{j} (\min(a_j, r_{jk}))$。
    * **特点**：类似于“木桶效应”的变体或“最大长板效应”。它只看最关键的那个指标，忽略次要因素。
    * *注：代码底层 `fuzzy_synthesis` 函数中**已预埋**了此功能 (`method='min_max'`)，未来可通过参数激活。*

3.  **乘积最大型 $M(\cdot, \lor)$**
    * **逻辑**：$b_k = \max_{j} (a_j \cdot r_{jk})$。
    * **特点**：关注权重与隶属度的乘积最大值。

## Step 3: 解模糊 (Defuzzification)

经过合成计算，我们对 George 的评价可能是一个模糊向量，例如 $B_{george} = [0.35, 0.45, 0.15, 0.05]$。
为了辅助决策，我们需要将这个向量转化为明确的结果。

主要有两个步骤（或两种路径）：

### 1. 最大隶属度原则 (Max Membership Principle)
* **操作**：直接看 $B$ 向量中哪个值最大。
* **示例**：在 $[0.35, \mathbf{0.45}, 0.15, 0.05]$ 中，0.45 最大，对应等级 $g_2$。
* **结论**：George 的评级为 **$g_2$**。
* *代码默认输出此结果。*

### 2. 加权得分法 (Weighted Score Method)
* **前提**：给每个等级赋予一个分值（Scores），例如 $S = [100, 80, 60, 40]$。
* **操作**：计算期望值。Score $= \sum B_k \cdot S_k$。
* **结论**：算出一个具体的数值（如 82.5 分），便于精细化排名。
* *代码底层 `defuzzify` 函数已预埋 `weighted_score` 接口。*

# 代码调用示例 (模拟主函数调用)
import numpy as np
import package_draft as pkg # 假设这是我们的算法包

# 1. 定义数据
weights = [0.10, 0.20, 0.30, 0.40]
criterion_names = ['weight', 'length', 'height', 'age']
criterion_type = ['max', 'max', 'max', 'max']
grade_names = ['g1', 'g2', 'g3', 'g4']
sample_names = ['george', 'marx', 'anna']

dataset = np.array([
    [9, 3, 4, 7],   # george
    [6, 3, 4, 6],   # marx 
    [4, 9, 7, 3],   # anna
])

# 2. 执行算法
# 注意：theme='green_purple' 是为了更好区分不同样本
result = pkg.fuzzy_comprehensive_method(
    criterion_names, 
    criterion_type, 
    grade_names, 
    dataset, 
    weights, 
    sample_names,
    graph_radar=True,   # 绘制雷达图
    graph_heat=True,    # 绘制热力图
    theme='green_purple',
    save_prefix='demo_output'
)

# 3. 打印核心结果
print("--- Defuzzified Results (等级判定) ---")
for res in result['Defuzzified_Results']:
    print(f"Sample: {sample_names[res['sample_index']]}, Best Grade: {res['best_grade']}, Confidence: {res['confidence']:.4f}")